# III. Pares candidatos y Autores similares

En esta parte idetificaremos pares candidatos de tweets y aplicaremos thresholding para definir autores similares.

In [2]:
import numpy as np
import pandas as pd
import collections
import itertools
import ast
import random

### Importar Trabajo previo

In [3]:
resumen = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Consolidar FHs a partir de los archivos guardados en disco.

In [4]:
FH = np.genfromtxt('fhs/file0.csv', delimiter=',')
for i in range(1, 10):
    FH2 = np.genfromtxt(f'fhs/file{i}.csv', delimiter=',')
    FH = np.concatenate((FH, FH2), axis=1)
FH.shape

(20, 203726)

## Banding

Mostrar distancia mínima a la que debe estar dos tweet para ser considerados pares candidatos

In [5]:
r = 5
b = 4
round((1/b)**(1/r), 3)

0.758

Ejecutar banding y obtener pares candidatos

In [6]:
# https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc
sig_mat = FH
n, d = sig_mat.shape
hashbuckets = collections.defaultdict(set)
bands = np.array_split(sig_mat, b, axis=0)
for i,band in enumerate(bands):
    for j in range(d):
        # The last value must be made a string, to prevent accidental
        # key collisions of r+1 integers when we really only want
        # keys of r integers plus a band index
        band_id = tuple(list(band[:,j])+[str(i)])
        hashbuckets[band_id].add(j)
candidate_pairs = set()
for bucket in hashbuckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)
print(f"Obtuvimos {len(candidate_pairs)} pares candidatos, con b = {b} y r = {r}")

Obtuvimos 2235158 pares candidatos, con b = 4 y r = 5


## Limpiar pares de candidatos

Quitar tweets idénticos de los pares similares

In [7]:
trimmed_cadidate_pairs = []
for pair in candidate_pairs:
        a, b = pair
        text_a = resumen.text[a]
        text_b = resumen.text[b]
        if (text_a != text_b) and ('http' not in text_a) and ('http' not in text_b):
                trimmed_cadidate_pairs.append((a, b))
print(f"Quedan {len(trimmed_cadidate_pairs)} pares candidatos")

Quedan 1035758 pares candidatos


Mostrar 3 pares de tweets presuntamente similares

In [8]:
for pair in random.sample(list(trimmed_cadidate_pairs), 3):
    a, b = pair
    print(f"1. {resumen.screen_name[a]}: ",resumen.text[a])
    print(f"2. {resumen.screen_name[b]}: ",resumen.text[b])
    print("----------")

1. Vicente22937071:  @gdominguez_ @MEQChile @ElisaLoncon @Jaime_Bassa @NoNeutrales @convencioncl #Rechazolacasacomunista
2. Indigente_pro:  @jaran_13 @gdominguez_ @MEQChile @ElisaLoncon @Jaime_Bassa @convencioncl pobre weon pasao a caca que quiere puros derechos el aweonao
----------
1. jorgevamaceda73:  @Skywalk09577817 @danielstingo En estos momentos, la mejor arma ofertas para #Rechazar.
2. albert_1917:  @Pasuchi2010 @danielstingo Estos wns tienen del día que les pidan #RechazoPopular
----------
1. Dpm_Chile2020:  No ➕ CORRUPCIÓN
#NuevaConstituciónParaChile #AprueboDeSalida #Apruebo4deSeptiembre
2. barryalva:  @berfontaine El apruebo le va a volar la raja al rechazo. #AprueboDeSalida #Apruebo4deSeptiembre #AprueboNuevaConstitucion
----------


## Identificar autores similares

Contar cuantos tweets similares hay para cada par de autores considerados candidatos similares

In [25]:
count_dict = dict()

def def_value():
    return 0
for pair in trimmed_cadidate_pairs:
    a, b = pair
    user_a = resumen.screen_name[a]
    user_b = resumen.screen_name[b]
    if user_a != user_b:
        try:
            count_dict[user_a][user_b] += 1
        except KeyError:
            count_dict[user_a] = collections.defaultdict(def_value) # No retorna key error
            count_dict[user_a][user_b] += 1
            
        try:
            count_dict[user_b][user_a] += 1
        except KeyError:
            count_dict[user_b] = collections.defaultdict(def_value) # No retorna key error
            count_dict[user_b][user_a] += 1
            

Mostrar solo los autores que están por sobre un threshold de tweets similares

In [52]:
count_df = pd.DataFrame(count_dict).reset_index()
count_long = pd.melt(count_df, id_vars='index')
similar_authors = count_long[count_long.value >= 100].rename({'index': 'user_1', 'variable': 'user_2', 'value':'sim_tweet_count'}, axis = 1)
similar_authors.sort_values("sim_tweet_count", ascending=False)

,user_1,user_2,sim_tweet_count
4488906,janelg88,Daniela_Platz,1850.0
4493799,Daniela_Platz,janelg88,1850.0
560181,x1educalidad,MEugeniaDiazJ,1478.0
637851,MEugeniaDiazJ,x1educalidad,1478.0
8289185,RomilioValenzu4,osotroncoso,944.0
...,...,...,...
10221077,RomilioValenzu4,caseoievski,105.0
1193136,gatitafuriosa4,RomilioValenzu4,103.0
17482661,RomilioValenzu4,gatitafuriosa4,103.0
1370500,MAJOCAPA196556,Natacha55538315,101.0


Función para imprimir tweets de dos candidatos pares:

In [47]:
def show_tweets_from_candidates(user_1, user_2, amount):
    iterations = 0
    for pair in trimmed_cadidate_pairs:
        a, b = pair
        user_a = resumen.screen_name[a]
        user_b = resumen.screen_name[b]
        if ((user_a == user_1) and (user_b == user_2)) or ((user_b == user_1) and (user_a == user_2)):
            print("By user ", user_a,":\n",  f'"{resumen.text[a]}"\n')
            print("By user ", user_b, ":\n", f'"{resumen.text[b]}"')
            print("_______________________________________________________\n")
            if iterations == (amount-1):
                break
            iterations+=1

Veamos tres ejemplos de pares de tweets de un par de autores similares.

In [48]:
show_tweets_from_candidates("Natacha55538315", "MAJOCAPA196556", 3)

By user  Natacha55538315 :
 "@Gonzanarko @tere_marinovic QUE TE PASA MUSICOO ! FRACASADOO ! GUEON PENCA ! UNA PLASTA ACEFALA ! QUE TE CREIII! CTM VENIR A GUEVIAR"

By user  MAJOCAPA196556 :
 "@SarahCo74398491 @tere_marinovic @Yohana_Barria @cata_1933 @FuerzaCarab @Carabdechile NAZI"
_______________________________________________________

By user  Natacha55538315 :
 "@Gonzanarko @tere_marinovic QUE TE PASA MUSICOO ! FRACASADOO ! GUEON PENCA ! UNA PLASTA ACEFALA ! QUE TE CREIII! CTM VENIR A GUEVIAR"

By user  MAJOCAPA196556 :
 "@SarahCo74398491 @facho1000 @tere_marinovic @Yohana_Barria @cata_1933 @FuerzaCarab @Carabdechile NAZI Y GAY ENCUBIERTO

K FOME TU VIDA GORDA"
_______________________________________________________

By user  MAJOCAPA196556 :
 "@facho1000 @SarahCo74398491 @tere_marinovic @Yohana_Barria @cata_1933 @FuerzaCarab @Carabdechile TRIPLE NAZI"

By user  Natacha55538315 :
 "@CamiloUribeP @tere_marinovic TU ERES EL TARADO BOTIJA LOVER ! DIGNO DE LASTIMA MURCIÉLAGO!!"
______

A continuación podemos examinar un cierto número de tweets para un n dado de autores similares.

In [51]:
def shown_n_similar_authors_examples(n_author_pairs, n_examples_per_pair):
    iteration = 0
    for index, row in similar_authors.iterrows():
        user_1 = row.user_1
        user_2 = row.user_2
        show_tweets_from_candidates(user_1, user_2, n_examples_per_pair) # Sacar 3 ejemplos por cada par
        if iteration == (n_author_pairs-1):
            break
        iteration += 1
shown_n_similar_authors_examples(2, 3)
# Se mostraran 3 ejemplos para 2 pares de autores similares: 

By user  aurora4310 :
 "@CamiRT7 @AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Se pico"

By user  JaggerGuti :
 "@Elinis_Benetash @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile @GAMBA_CL O empiezo yo? 😂"
_______________________________________________________

By user  JaggerGuti :
 "@Elinis_Benetash @sotan_n @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Propiedad y digna... 😂 😂 😂"

By user  aurora4310 :
 "@AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile No tienen ningún poder"
_______________________________________________________

By user  JaggerGuti :
 "@Elinis_Benetash @sotan_n @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @danielstingo @MEQChile Propiedad y digna... 😂 😂 😂"

By user  aurora4310 :
 "@AprendizCrypto @pillan197811 @fernando_atria @DajacJara @AXELKAISER @rocicantuarias @dani